In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import random
import os
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.decomposition import PCA

In [ ]:
def get_minidataset(data_size = 10000):
  train_features_temp = pd.read_csv("../input/lish-moa/train_features.csv")
  train_targets_temp = pd.read_csv("../input/lish-moa/train_targets_scored.csv")

  # full_data = pd.concat([train_features_temp, train_targets_temp], axis=1)

  # Creating the whole dataset by joining
  full_data = train_features_temp.join(train_targets_temp.set_index('sig_id'), on='sig_id')
  # print(full_data.head())
  # print(full_data.columns)

  sample_df = full_data.sample(n=data_size)
  train_features_temp = sample_df[ train_features_temp.columns]
  train_targets_temp = sample_df[train_targets_temp.columns]

  COLS = ['cp_type','cp_dose']
  FE = []
  for col in COLS:
      for mod in train_features_temp[col].unique():
          FE.append(mod)
          train_features_temp[mod] = (train_features_temp[col] == mod).astype(int)
  del train_features_temp['sig_id']
  del train_features_temp['cp_type']
  del train_features_temp['cp_dose']
  FE+=list(train_features_temp.columns) 
  del train_targets_temp['sig_id']

  scaler = MinMaxScaler()
  train_features_temp_rescaled = scaler.fit_transform(train_features_temp)

  return (train_features_temp_rescaled,train_targets_temp)
  # comparison_column = np.where(full_data["col1"] == full_data["col2"], True, False)

# train_sample, test_sample = get_minidataset()
# print(train_sample)
# print(test_sample)

In [ ]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(42)

In [ ]:
train_features = pd.read_csv("../input/lish-moa/train_features.csv")
test_features = pd.read_csv("../input/lish-moa/test_features.csv")
train_targets = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
no_score = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
ss = pd.read_csv("../input/lish-moa/sample_submission.csv")

In [ ]:
COLS = ['cp_type','cp_dose']
FE = []
print(train_features.head())
# print("cols:", train_features['cp_type'].unique())
# print("FE Before", FE)
for col in COLS:
    for mod in train_features[col].unique():
        FE.append(mod)
        train_features[mod] = (train_features[col] == mod).astype(int)
del train_features['sig_id']
del train_features['cp_type']
del train_features['cp_dose']
# print("FE Mid", FE)
FE+=list(train_features.columns) 
del train_targets['sig_id']
# print("FE After", FE)
# print(train_features.head())
train_targets_copy = train_targets.copy()


for col in COLS:
    for mod in test_features[col].unique():
        test_features[mod] = (test_features[col] == mod).astype(int)
sig_id = pd.DataFrame()
sig_id = test_features.pop('sig_id')
del test_features['cp_type']
del test_features['cp_dose']

In [ ]:
train_features.describe(percentiles = [.25,.5,.75,.95,.97])

In [ ]:
print(train_features.var())

In [ ]:
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(train_features)

# Add code for rescaling test set also here!
data_rescaled_test = scaler.fit_transform(test_features)

print(data_rescaled)
print(data_rescaled_test)
# data_rescaled_df = pd.DataFrame(data_rescaled)
# print(data_rescaled_df.var())

In [ ]:
pca = PCA(n_components = 0.97)
pca.fit(data_rescaled)
reduced_train = pca.transform(data_rescaled)
reduced_test =  pca.transform(data_rescaled_test)

reduced_train_df = pd.DataFrame(reduced_train)
reduced_test_df = pd.DataFrame(reduced_test)

print(reduced_train_df)
print(reduced_test_df)

In [ ]:
# Model for .97 PCA
def make_model(optimizer='Adamax', init_mode = 'uniform', activation = 'sigmoid' , neurons = 2445):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(602),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(units=neurons, kernel_initializer=init_mode, activation=activation),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=neurons, kernel_initializer=init_mode, activation=activation),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(206,kernel_initializer=init_mode, activation="sigmoid")
    ])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy", "AUC"])
    return model

In [ ]:
NFOLD = 5
kf = KFold(n_splits=NFOLD)

BATCH_SIZE= 32
EPOCHS= 60
# print(test_features)

pe = np.zeros((reduced_test_df.shape[0], 206))

# train_features = train_features.values
train_features = reduced_train
test_features = reduced_test
train_targets = train_targets.values
pred = np.zeros((reduced_train_df.shape[0], 206))

cnt=0
for tr_idx, val_idx in kf.split(train_features):
#     reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
    cnt += 1
    print(f"FOLD {cnt}")
    net = make_model()
    net.fit(train_features[tr_idx], train_targets[tr_idx], batch_size=BATCH_SIZE, epochs=EPOCHS, 
            validation_data=(train_features[val_idx], train_targets[val_idx]), verbose=0)
    print("train", net.evaluate(train_features[tr_idx], train_targets[tr_idx], verbose=0, batch_size=BATCH_SIZE))
    print("val", net.evaluate(train_features[val_idx], train_targets[val_idx], verbose=0, batch_size=BATCH_SIZE))
    print("predict val...")
    pred[val_idx] = net.predict(train_features[val_idx], batch_size=BATCH_SIZE, verbose=0)
    print("predict test...")
    pe += net.predict(test_features, batch_size=BATCH_SIZE, verbose=0) / NFOLD

In [ ]:
pe.shape

columns = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
del columns['sig_id']
sub = pd.DataFrame(data=pe, columns=columns.columns)

In [ ]:
def metric(y_true, y_pred):
    metrics = []
    for _target in y_true.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)
pred
train_targets_copy
print(metric(pd.DataFrame(train_targets),pd.DataFrame(pred)))

In [ ]:
sub.insert(0, column = 'sig_id', value=ss['sig_id'])

In [ ]:
sub.to_csv('submission.csv', index=False)
